In [2]:
import cv2

In [4]:
!apt update
!apt install -y cmake
!pip install dlib

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,533 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13

In [5]:
import dlib
import numpy as np

In [17]:
!wget   http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 # DOWNLOAD LINK
!bunzip2 /content/shape_predictor_68_face_landmarks.dat.bz2
datFile =  "/content/shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(datFile)

--2022-08-23 11:04:15--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  88.2MB/s    in 0.7s    

2022-08-23 11:04:15 (88.2 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]

bunzip2: Output file /content/shape_predictor_68_face_landmarks.dat already exists.


In [18]:
detector = dlib.get_frontal_face_detector()

In [25]:
def get_landmark(im):
  rects = detector(im, 1)
  if len(rects) > 1:
    return "error"
  if len(rects) == 0:
    return "erroe"
  return np.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()])

def landmark_annotate(im, landmarks):
  im = im.copy()
  for idx, point in enumerate(landmarks):
    pos = (point[0,0], point[0,1])
    cv2.putText(im, str(idx), pos, fontFace = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, font_Scale=0.4, color = (1,2,255))
    cv2.circle(im, pos, 3, color = (0,2,2))
  return im

In [20]:
def get_left_lip_edge(landmarks):
  left_lip_edge = landmarks[61]
  return int(left_lip_edge)
def get_right_lip_edge(landmarks): 
  right_lip_edge = landmarks[65]
  return int(right_lip_edge)

In [21]:
def get_lip_distance(image):
  landmarks = get_landmark(image)
  if landmarks == "error":
    return image, 0
  image_with_landmarks = landmark_annotate(image, landmarks)
  left_edge = get_left_lip_edge(landmarks)
  right_edge = get_right_lip_edge(landmarks)
  lip_distance = abs(right_edge - left_edge)
  return image_with_landmarks, lip_distance

In [26]:
cap = cv2.VideoCapture(0)
wide_mouth = 0
wide_mouth_status = False
while True:
  ret, frame = cap.read()
  image_landmarks, lip_distance = get_lip_distance(frame)
  previous_wide_mouth_status = wide_mouth_status
  if lip_distance > 3:
    wide_mouth_status = True
    cv2.putText(frame, "mouth wide open", (50, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
    wide_mouth_count = "wide mouth count" + str(wide_mouth + 1)
    cv2.putText(frame, wide_mouth_count, (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,127), 2)
  else:
    wide_mouth_status = False
  if previous_wide_mouth_status == True and wide_mouth_status == False:
    wide_mouth+=1
  cv2.imshow('live edges', landmarks[61], landmarks[65])
  cv2.imshow('mouth wide open detection', frame)
  if cv2.waitKey(1) == 20:
    break
cap.release()
cv2.destroyAllWindows()

TypeError: ignored